In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6

In [ ]:
tr = pd.read_csv('input/train.csv')

In [ ]:
te = pd.read_csv('input/test.csv')
te['TripType'] = 0

In [ ]:
d = pd.concat([tr, te])
d.TripType = d.TripType.astype(int)
del d['FinelineNumber']
del d['Upc']

In [ ]:
e = pd.get_dummies(d, columns=['DepartmentDescription', 'Weekday', 'TripType'], dummy_na=True)
e['Weekday'] = d.Weekday
e['TripType'] = d.TripType
#e.info()

In [ ]:
def sanitize(s):
    ret = s.replace(',', '').replace('&', 'AND').replace('/', ' ').replace(' ', '_')
    ret = ret.split('.')[0]
    return ret
e.columns = [sanitize(c) for c in e.columns]

In [ ]:
for c in e.columns:
    if c.startswith('DepartmentDescription_'):
        e[c] = e.ScanCount * e[c]

In [ ]:
def agg_visit(g):
    vn = int(g.VisitNumber.min())
    if vn % 10000 == 1:
        print vn
    scp = int(g[g.ScanCount > 0].ScanCount.sum())
    scn = -int(g[g.ScanCount < 0].ScanCount.sum())
    sc = scp + scn
    ret = DataFrame({
            'ScanCountPos': [scp],
            'ScanCountNeg': [scn],
            'ScanCountPosRatio': [1.0*scp/sc],
            'ScanCountNegRatio': [1.0*scn/sc],
            'TripType': [int(g.TripType.min())],
            'Weekday': [g.Weekday.min()]
        })
    for c in g.columns:
        if c.startswith('DepartmentDescription_'):
            ret[c + "_count"] = g[c].sum()
            ret[c + "_ratio"] = g[c].sum() / sc
    for c in g.columns:
        if c.startswith('Weekday_'):
            ret[c] = [int(g[c].min())]
    for c in g.columns:
        if c.startswith('TripType_'):
            ret[c] = [int(g[c].min())]
    return ret
g = e.groupby(by=['VisitNumber'])
h = g.apply(agg_visit)
h['VisitNumber'] = h.index.get_level_values(0)

In [ ]:
del h['TripType_nan']

In [ ]:
h.to_csv('tmp/features_02.csv', index=False)